In [ ]:
%pip install pandas_ta

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 8.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 10.9 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [175]:
# Install missing package in the notebook environment


import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [216]:
df_BTC = pd.read_csv('data/BTCUSDT_15m.csv')
df_DOGE = pd.read_csv('data/DOGEUSDT_15m.csv')

In [202]:
df_BTC = pd.read_csv('data_Jack_bianance/15min/BTCUSDT_15m.csv')
df_DOGE = pd.read_csv('data_Jack_bianance/15min/DOGEUSDT_15m.csv')

# Trend Detection

### Calculate SMA

In [217]:
def calculate_SMA20(df, column_name='close', period = 20):

    return df[column_name].rolling(window=period).mean()

def calculate_SMA50(df, column_name='close', period = 50):
    
    return df[column_name].rolling(window=period).mean()

### Calculate EMA

In [218]:
def calculate_EMA20(df, column_name: str = 'close', period: int = 20):

    return df[column_name].ewm(span=period, adjust=False).mean()

def calculate_EMA50(df, column_name: str = 'close', period: int = 50):

    return df[column_name].ewm(span=period, adjust=False).mean()

In [219]:
df_BTC['SMA_20'] = calculate_SMA20(df_BTC)
df_BTC['SMA_50'] = calculate_SMA50(df_BTC)

df_BTC['EMA_20'] = calculate_EMA20(df_BTC)
df_BTC['EMA_50'] = calculate_EMA50(df_BTC)

print(df_BTC)
print(type(df_BTC))

                       timestamp       open       high        low      close  \
0      2025-01-01 00:00:00+00:00   93576.00   93702.15   93489.03   93656.18   
1      2025-01-01 00:15:00+00:00   93656.19   93898.05   93614.95   93761.90   
2      2025-01-01 00:30:00+00:00   93761.90   93963.67   93761.89   93885.01   
3      2025-01-01 00:45:00+00:00   93885.00   94509.42   93864.44   94401.14   
4      2025-01-01 01:00:00+00:00   94401.13   94408.72   93880.00   94153.05   
...                          ...        ...        ...        ...        ...   
29988  2025-11-09 09:00:00+00:00  101972.94  102112.01  101961.18  102044.64   
29989  2025-11-09 09:15:00+00:00  102044.64  102127.26  101881.74  101962.81   
29990  2025-11-09 09:30:00+00:00  101962.81  102035.94  101905.66  101905.82   
29991  2025-11-09 09:45:00+00:00  101905.83  101905.83  101631.62  101657.46   
29992  2025-11-09 10:00:00+00:00  101657.46  101664.91  101500.01  101584.85   

          volume       SMA_20       SMA

In [220]:
# --- Trend Detection Column ---
def detect_trend_series(df, sma_short='SMA_20', sma_long='SMA_50', window=5):
    trend_list = []

    for i in range(len(df)):
        if i < window:
            trend_list.append(np.nan)
        else:
            recent = df.iloc[i-window+1:i+1]
            if all(recent[sma_short] > recent[sma_long]):
                trend_list.append("bullish")
            elif all(recent[sma_short] < recent[sma_long]):
                trend_list.append("bearish")
            else:
                trend_list.append("volatile")
    
    return trend_list



In [221]:
df_BTC['trend'] = detect_trend_series(df_BTC)

print(df_BTC[['close', 'SMA_20', 'SMA_50', 'trend']].tail(30))

           close       SMA_20       SMA_50     trend
29963  101633.25  102076.4725  101994.8706   bullish
29964  101712.42  102044.0730  101990.5990   bullish
29965  101505.39  102002.8660  101984.1630   bullish
29966  101642.41  101972.6140  101980.0112  volatile
29967  101662.05  101936.8435  101981.4138  volatile
29968  101826.79  101909.2160  101986.5880  volatile
29969  102070.90  101890.2125  101993.7770  volatile
29970  101879.69  101865.5980  101992.8632   bearish
29971  101724.91  101836.1965  101990.5380   bearish
29972  101792.34  101823.2510  101991.6162   bearish
29973  101982.79  101833.4335  101994.2676   bearish
29974  101808.00  101823.0415  101997.4562   bearish
29975  101928.54  101829.5815  102000.0170   bearish
29976  101879.99  101825.7080  101999.8238   bearish
29977  101887.48  101827.7815  101997.6830   bearish
29978  101944.35  101825.0495  101996.7212   bearish
29979  101752.83  101810.4375  101991.5932   bearish
29980  101801.73  101806.8230  101986.7846   b

# Find Pivot Points

In [222]:
def find_pivots(df: pd.DataFrame, window: int = 2, min_gap: int = 1) -> pd.Series:
    """
    Detect pivot (fractal) points in OHLC data, then thin so that in any
    `min_gap` consecutive bars there is at most one pivot (of any type).

    Returns:
        pd.Series: 0 = no pivot, 1 = pivot high, 2 = pivot low, 3 = both.
    """
    pivots = [0] * len(df)

    for candle in range(len(df)):
        # skip edges
        if candle - window < 0 or candle + window >= len(df):
            continue

        pivotHigh = True
        pivotLow = True

        # check neighborhood [candle-window, candle+window]
        for i in range(candle - window, candle + window + 1):
            if df.iloc[candle].low > df.iloc[i].low:
                pivotLow = False
            if df.iloc[candle].high < df.iloc[i].high:
                pivotHigh = False

        if pivotHigh and pivotLow:
            pivots[candle] = 3 # both pivot high and low (rare case) 
        elif pivotHigh:
            pivots[candle] = 1  # pivot high
        elif pivotLow:
            pivots[candle] = 2  # pivot low

    # === Spacing enforcement: at most one pivot in any `min_gap` bars ===
    # Treat any non-zero (1,2,3) as a pivot; keep the first, drop others within the next (min_gap-1) bars.
    piv = np.array(pivots, dtype=int)
    last_kept = -10**9  # sufficiently negative

    # indices of any pivot (1,2,3), in chronological order
    pivot_idx = np.where(piv != 0)[0]

    for idx in pivot_idx:
        if idx - last_kept >= min_gap:
            last_kept = idx      # keep this pivot
        else:
            piv[idx] = 0         # too close to previous pivot → drop

    return pd.Series(piv, index=df.index)



In [223]:
df_DOGE['isPivot'] = find_pivots(df_DOGE)

# Finding Support + Setup

In [224]:
import numpy as np
import pandas as pd

# ---------- utilities ----------
def prep_df(df: pd.DataFrame, ts_col: str = 'timestamp') -> pd.DataFrame:
    out = df.copy()
    if ts_col in out.columns:
        if not pd.api.types.is_datetime64_any_dtype(out[ts_col]):
            out[ts_col] = pd.to_datetime(out[ts_col], errors='coerce')
        if pd.api.types.is_datetime64tz_dtype(out[ts_col].dtype):
            out[ts_col] = out[ts_col].dt.tz_convert('UTC').dt.tz_localize(None)
    return out

def compute_pivots_if_missing(df: pd.DataFrame, window: int, min_gap: int, pivot_col='isPivot') -> pd.DataFrame:
    out = df.copy()
    if pivot_col not in out.columns:
        out[pivot_col] = find_pivots(out, window=window, min_gap=min_gap)  # uses your function
    return out

def within_pct(a: float, b: float, pct: float) -> bool:
    if b == 0 or np.isnan(a) or np.isnan(b): return False
    return abs(a - b) / b <= pct

# ---------- Step 1: find support/resistance levels from pairs of close pivots ----------
def find_close_pivot_levels(df: pd.DataFrame,
                            tol_pct: float = 0.0025,   # 0.25%
                            pivot_col: str = 'isPivot'):
    """
    Find levels by pairing consecutive pivots of the SAME type that are within tol_pct in price.
    - Support: two pivot LOWS within tol_pct -> level = average(low1, low2)
    - Resistance: two pivot HIGHS within tol_pct -> level = average(high1, high2)
    Returns: lists of dicts for supports and resistances.
    """
    supports, resistances = [], []

    # collect indices by type (strict; treat code 3 as both)
    low_idxs = df.index[df[pivot_col].isin([2, 3])].to_list()
    high_idxs = df.index[df[pivot_col].isin([1, 3])].to_list()

    # pair consecutive lows
    for i in range(len(low_idxs) - 1):
        i1, i2 = low_idxs[i], low_idxs[i+1]
        p1, p2 = float(df.at[i1, 'low']), float(df.at[i2, 'low'])
        if within_pct(p2, p1, tol_pct):
            lvl = (p1 + p2) / 2.0
            supports.append({
                'level': lvl,
                'first_idx': df.index.get_loc(i1),
                'second_idx': df.index.get_loc(i2),
                'first_time': df.at[i1, 'timestamp'] if 'timestamp' in df.columns else i1,
                'second_time': df.at[i2, 'timestamp'] if 'timestamp' in df.columns else i2,
                'first_price': p1,
                'second_price': p2
            })

    # pair consecutive highs
    for i in range(len(high_idxs) - 1):
        i1, i2 = high_idxs[i], high_idxs[i+1]
        p1, p2 = float(df.at[i1, 'high']), float(df.at[i2, 'high'])
        if within_pct(p2, p1, tol_pct):
            lvl = (p1 + p2) / 2.0
            resistances.append({
                'level': lvl,
                'first_idx': df.index.get_loc(i1),
                'second_idx': df.index.get_loc(i2),
                'first_time': df.at[i1, 'timestamp'] if 'timestamp' in df.columns else i1,
                'second_time': df.at[i2, 'timestamp'] if 'timestamp' in df.columns else i2,
                'first_price': p1,
                'second_price': p2
            })

    return supports, resistances

# ---------- helper: previous pivot high before a given absolute row index ----------
def prev_pivot_high_before(df: pd.DataFrame, abs_row: int, pivot_col: str = 'isPivot'):
    # abs_row is positional index (0..len-1). Convert to actual df index label:
    for pos in range(abs_row - 1, -1, -1):
        label = df.index[pos]
        piv = int(df.at[label, pivot_col])
        if piv in (1, 3):
            return pos, float(df.at[label, 'high'])
    return None, None

# ---------- Step 2 (for supports): scan window for PL1 then PH2 confirmation ----------
def find_setup_for_support(df: pd.DataFrame,
                           support_level: float,
                           start_pos: int,     # start from the LATER pivot's position
                           n: int = 20,
                           far_pct: float = 0.01,    # 1%
                           pivot_col: str = 'isPivot'):
    """
    In the next n bars starting at start_pos:
      1) Find PL1: a pivot low >2% below support.
      2) Let PH1 be the pivot high before PL1.
      3) Find PH2 after PL1 such that PH2 > PH1 (within the same n-bar window).
      If found: return dict with PL1, PH2, buy_price at 0.618 retracement.
    Otherwise: return None.
    """
    end_pos = min(len(df) - 1, start_pos + n)
    pl1_pos = None
    pl1_price = None

    # 1) find PL1
    for pos in range(start_pos, end_pos + 1):
        label = df.index[pos]
        piv = int(df.at[label, pivot_col])
        if piv in (2, 3):
            low_px = float(df.at[label, 'low'])
            if (support_level - low_px) / support_level > far_pct:
                pl1_pos, pl1_price = pos, low_px
                break

    if pl1_pos is None:
        return None  # no break below support

    # 2) PH1: pivot high immediately before PL1
    ph1_pos, ph1_price = prev_pivot_high_before(df, pl1_pos, pivot_col=pivot_col)
    if ph1_pos is None or ph1_price is None:
        return None  # cannot define PH1

    # 3) find PH2 after PL1 that exceeds PH1
    for pos in range(pl1_pos + 1, end_pos + 1):
        label = df.index[pos]
        piv = int(df.at[label, pivot_col])
        if piv in (1, 3):
            high_px = float(df.at[label, 'high'])
            if high_px > ph1_price:
                ph2_pos, ph2_price = pos, high_px
                # Fibonacci buy at 0.618 retracement of (PL1 -> PH2)
                buy_price = ph2_price - 0.618 * (ph2_price - pl1_price)
                return {
                    'support_level': support_level,
                    'start_pos': start_pos,
                    'end_pos': pos,
                    'pl1_pos': pl1_pos,
                    'pl1_price': pl1_price,
                    'ph1_pos': ph1_pos,
                    'ph1_price': ph1_price,
                    'ph2_pos': ph2_pos,
                    'ph2_price': ph2_price,
                    'buy_price_0618': buy_price,
                    'tradable_setup': 1
                }

    return None

# ---------- Orchestrator ----------
def run_liquidity_gap_fvg(df: pd.DataFrame,
                          tol_pct_level: float = 0.0025,  # 0.5% for defining level
                          n_window: int = 20,
                          far_pct_break: float = 0.01,   # 1% break
                          pivot_col: str = 'isPivot'):
    """
    Implements your new logic for both Step 1 and Step 2 (support side).
    Returns:
      out_df: input with helper columns,
      levels: dict with 'supports' and 'resistances',
      setups: list of detected setups (support path)
    """
    df = prep_df(df)
    df = compute_pivots_if_missing(df, 2, 1 ,pivot_col=pivot_col)

    # Step 1: find support/resistance from close pivot pairs
    supports, resistances = find_close_pivot_levels(df, tol_pct=tol_pct_level, pivot_col=pivot_col)

    # For each support, Step 2: look ahead n bars from the later pivot of the pair
    setups = []
    for s in supports:
        start_pos = s['second_idx']  # later pivot’s position
        res = find_setup_for_support(
            df,
            support_level=s['level'],
            start_pos=start_pos,
            n=n_window,
            far_pct=far_pct_break,
            pivot_col=pivot_col
        )
        if res:
            # attach some provenance from the level
            res['support_first_pos']  = s['first_idx']
            res['support_second_pos'] = s['second_idx']
            res['support_first_price']  = s['first_price']
            res['support_second_price'] = s['second_price']
            setups.append(res)

    # Optional: convenience columns for quick plotting/inspection (Debugging)
    out = df.copy()
    out['close_pivot_support'] = np.nan
    out['close_pivot_resist']  = np.nan
    for s in supports:
        out.at[out.index[s['second_idx']], 'close_pivot_support'] = s['level']
    for r in resistances:
        out.at[out.index[r['second_idx']], 'close_pivot_resist'] = r['level']

    levels = {'supports': supports, 'resistances': resistances}
    return out, levels, setups

In [225]:
# Ensure your df (e.g., df_BTC) already has OHLC + timestamp
df_DOGE = prep_df(df_DOGE)
df_DOGE = compute_pivots_if_missing(df_DOGE, window=2, min_gap=1)

out_df, levels, setups = run_liquidity_gap_fvg(
    df_DOGE,
    tol_pct_level=0.0025,   # ±0.25% to define support/resistance via two close pivots
    n_window=20,           # scan window after support
    far_pct_break=0.01,    # PL1 must be >1% below support
    pivot_col='isPivot'
)

print(f"Supports found: {len(levels['supports'])}, Resistances found: {len(levels['resistances'])}")
print(f"Tradable setups found: {len(setups)}")
if setups:
    print(setups[-1])
    # Buy price (0.618)
    print("Buy @ 0.618:", setups[-1]['buy_price_0618'])

Supports found: 1108, Resistances found: 1314
Tradable setups found: 45
{'support_level': 0.162685, 'start_pos': 29700, 'end_pos': 29719, 'pl1_pos': 29715, 'pl1_price': 0.16098, 'ph1_pos': 29710, 'ph1_price': 0.16336, 'ph2_pos': 29719, 'ph2_price': 0.16384, 'buy_price_0618': 0.16207252000000003, 'tradable_setup': 1, 'support_first_pos': 29693, 'support_second_pos': 29700, 'support_first_price': 0.16275, 'support_second_price': 0.16262}
Buy @ 0.618: 0.16207252000000003


/var/folders/p8/by67wr7139zdvwbp6501h9lh0000gn/T/ipykernel_64378/2048255335.py:10: DeprecationWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.



In [226]:
import plotly.graph_objects as go
import numpy as np

def plot_liquidity_fvg_window(out_df, setups, supports, row_start=None, row_end=None,
                              title="DOGE — Liquidity Gap / FVG Strategy"):
    df_full = out_df.sort_values('timestamp')
    if row_start is None: row_start = 0
    if row_end   is None: row_end   = len(df_full)
    dfp = df_full.iloc[row_start:row_end]   # DO NOT reset_index

    # Pivot markers on the visible slice
    piv_high = np.where(dfp['isPivot'].isin([1,3]), dfp['high'] * 1.001, np.nan)
    piv_low  = np.where(dfp['isPivot'].isin([2,3]), dfp['low']  * 0.999, np.nan)

    # Visible time bounds
    t_min = dfp['timestamp'].min()
    t_max = dfp['timestamp'].max()

    # Helpers
    def _pos_to_time(pos):
        if pos is None: return None
        pos = int(pos)
        if 0 <= pos < len(df_full):
            return df_full.iloc[pos]['timestamp']
        return None

    def _clip_segment(a, b, lo, hi):
        """Return clipped [x0,x1] within [lo,hi], or (None,None) if no overlap."""
        x0, x1 = min(a, b), max(a, b)
        x0c, x1c = max(x0, lo), min(x1, hi)
        return (x0c, x1c) if x0c <= x1c else (None, None)

    fig = go.Figure()

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=dfp['timestamp'],
        open=dfp['open'], high=dfp['high'],
        low=dfp['low'], close=dfp['close'],
        name='Candles'
    ))

    # Pivot highs / lows
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_high,
        mode='markers', name='Pivot High',
        marker=dict(symbol='triangle-up', color='red', size=8)
    ))
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_low,
        mode='markers', name='Pivot Low',
        marker=dict(symbol='triangle-down', color='lime', size=8)
    ))

    # Support lines ONLY between the two pivots (clipped to visible window)
    if supports:
        for s in supports:
            # prefer stored times; fall back to positions if needed
            t1 = s.get('first_time')
            t2 = s.get('second_time')
            if t1 is None or t2 is None:
                t1 = _pos_to_time(s.get('first_idx'))
                t2 = _pos_to_time(s.get('second_idx'))
            if t1 is None or t2 is None:
                continue

            x0, x1 = _clip_segment(t1, t2, t_min, t_max)
            if x0 is None:  # no overlap with window
                continue

            lvl = float(s['level'])
            fig.add_trace(go.Scatter(
                x=[x0, x1],
                y=[lvl, lvl],
                mode='lines',
                line=dict(color='green', dash='dot', width=1.8),
                name='Support',
                hovertemplate=f"Support {lvl:.2f}<extra></extra>",
                showlegend=False  # avoid legend spam for many supports
            ))

    # BUY points (0.618 Fib at PH2 time)
    if setups:
        buy_times, buy_prices = [], []
        for st in setups:
            ph2_time = st.get('ph2_time') or _pos_to_time(st.get('ph2_pos'))
            buy_price = st.get('buy_price_0618')
            if ph2_time is None or buy_price is None:
                continue
            if t_min <= ph2_time <= t_max:
                buy_times.append(ph2_time)
                buy_prices.append(buy_price)

                # Optional: PL1↔PH2 fib box for context
                pl1_time = st.get('pl1_time') or _pos_to_time(st.get('pl1_pos'))
                pl1_price = st.get('pl1_price'); ph2_price = st.get('ph2_price')
                if pl1_time and pl1_price and ph2_price:
                    x0, x1 = _clip_segment(pl1_time, ph2_time, t_min, t_max)
                    if x0 is not None:
                        fig.add_shape(
                            type="rect", xref="x", yref="y",
                            x0=x0, x1=x1,
                            y0=min(pl1_price, ph2_price), y1=max(pl1_price, ph2_price),
                            line=dict(color="gold", width=1, dash="dot"),
                            fillcolor="rgba(255,215,0,0.05)"
                        )

        if buy_times:
            fig.add_trace(go.Scatter(
                x=buy_times, y=buy_prices,
                mode='markers+text',
                name='Buy (0.618 Fib)',
                text=["BUY"] * len(buy_times),
                textposition="bottom center",
                marker=dict(symbol='star', size=12, color='gold'),
                hovertemplate="BUY @ 0.618 Fib<br>%{x|%Y-%m-%d %H:%M}<br>Price: %{y:.2f}<extra></extra>"
            ))

    fig.update_layout(
        title=title,
        xaxis_title="Time", yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
        plot_bgcolor="white"
    )
    return fig

# Plotting to Check

In [227]:
import plotly.graph_objects as go
import numpy as np

def plot_liquidity_fvg_window(out_df, setups, supports, row_start=None, row_end=None,
                              title="BTC — Liquidity Gap / FVG Strategy"):
    df_full = out_df.sort_values('timestamp')
    if row_start is None: row_start = 0
    if row_end   is None: row_end   = len(df_full)
    dfp = df_full.iloc[row_start:row_end]   # DO NOT reset_index

    # Pivot markers on the visible slice
    piv_high = np.where(dfp['isPivot'].isin([1,3]), dfp['high'] * 1.001, np.nan)
    piv_low  = np.where(dfp['isPivot'].isin([2,3]), dfp['low']  * 0.999, np.nan)

    # Visible time bounds
    t_min = dfp['timestamp'].min()
    t_max = dfp['timestamp'].max()

    # Helpers
    def _pos_to_time(pos):
        if pos is None: return None
        pos = int(pos)
        if 0 <= pos < len(df_full):
            return df_full.iloc[pos]['timestamp']
        return None

    def _clip_segment(a, b, lo, hi):
        """Return clipped [x0,x1] within [lo,hi], or (None,None) if no overlap."""
        x0, x1 = min(a, b), max(a, b)
        x0c, x1c = max(x0, lo), min(x1, hi)
        return (x0c, x1c) if x0c <= x1c else (None, None)

    fig = go.Figure()

    # Candlesticks
    fig.add_trace(go.Candlestick(
        x=dfp['timestamp'],
        open=dfp['open'], high=dfp['high'],
        low=dfp['low'], close=dfp['close'],
        name='Candles'
    ))

    # Pivot highs / lows
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_high,
        mode='markers', name='Pivot High',
        marker=dict(symbol='triangle-up', color='red', size=8)
    ))
    fig.add_trace(go.Scatter(
        x=dfp['timestamp'], y=piv_low,
        mode='markers', name='Pivot Low',
        marker=dict(symbol='triangle-down', color='lime', size=8)
    ))

    # Support lines ONLY between the two pivots (clipped to visible window)
    if supports:
        for s in supports:
            # prefer stored times; fall back to positions if needed
            t1 = s.get('first_time')
            t2 = s.get('second_time')
            if t1 is None or t2 is None:
                t1 = _pos_to_time(s.get('first_idx'))
                t2 = _pos_to_time(s.get('second_idx'))
            if t1 is None or t2 is None:
                continue

            x0, x1 = _clip_segment(t1, t2, t_min, t_max)
            if x0 is None:  # no overlap with window
                continue

            lvl = float(s['level'])
            fig.add_trace(go.Scatter(
                x=[x0, x1],
                y=[lvl, lvl],
                mode='lines',
                line=dict(color='green', dash='dot', width=1.8),
                name='Support',
                hovertemplate=f"Support {lvl:.2f}<extra></extra>",
                showlegend=False  # avoid legend spam for many supports
            ))

    # BUY points (0.618 Fib at PH2 time)
    if setups:
        buy_times, buy_prices = [], []
        for st in setups:
            ph2_time = st.get('ph2_time') or _pos_to_time(st.get('ph2_pos'))
            buy_price = st.get('buy_price_0618')
            if ph2_time is None or buy_price is None:
                continue
            if t_min <= ph2_time <= t_max:
                buy_times.append(ph2_time)
                buy_prices.append(buy_price)

                # Optional: PL1↔PH2 fib box for context
                pl1_time = st.get('pl1_time') or _pos_to_time(st.get('pl1_pos'))
                pl1_price = st.get('pl1_price'); ph2_price = st.get('ph2_price')
                if pl1_time and pl1_price and ph2_price:
                    x0, x1 = _clip_segment(pl1_time, ph2_time, t_min, t_max)
                    if x0 is not None:
                        fig.add_shape(
                            type="rect", xref="x", yref="y",
                            x0=x0, x1=x1,
                            y0=min(pl1_price, ph2_price), y1=max(pl1_price, ph2_price),
                            line=dict(color="gold", width=1, dash="dot"),
                            fillcolor="rgba(255,215,0,0.05)"
                        )

        if buy_times:
            fig.add_trace(go.Scatter(
                x=buy_times, y=buy_prices,
                mode='markers+text',
                name='Buy (0.618 Fib)',
                text=["BUY"] * len(buy_times),
                textposition="bottom center",
                marker=dict(symbol='star', size=12, color='gold'),
                hovertemplate="BUY @ 0.618 Fib<br>%{x|%Y-%m-%d %H:%M}<br>Price: %{y:.2f}<extra></extra>"
            ))

    fig.update_layout(
        title=title,
        xaxis_title="Time", yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='left', x=0),
        plot_bgcolor="white"
    )
    return fig

In [228]:
# Use FULL out_df, not the reset-index slice
fig = plot_liquidity_fvg_window(
    out_df,                       # full df from run_liquidity_gap_fvg
    setups=setups,                # setups list from run_liquidity_gap_fvg
    supports=levels['supports'],  # supports list from run_liquidity_gap_fvg
    row_start=29968, row_end=29993  # pick your window safely
)
fig.show()

# BackTesting

In [172]:
import numpy as np
import pandas as pd

def _find_entry_after_ph2(df: pd.DataFrame, ph2_pos: int, entry_price: float):
    """
    Scan forward from ph2_pos for the first bar whose range includes entry_price.
    Returns (entry_pos, entry_time) or (None, None) if never touched.
    """
    n = len(df)
    for pos in range(ph2_pos + 1, n):
        low = float(df.iloc[pos]['low'])
        high = float(df.iloc[pos]['high'])
        if low <= entry_price <= high:
            t = df.iloc[pos]['timestamp'] if 'timestamp' in df.columns else pos
            return pos, t
    return None, None

def _simulate_trade_long(df: pd.DataFrame,
                         entry_pos: int,
                         entry_price: float,
                         pl1_price: float,
                         support_level: float,
                         ph2_price: float):
    """
    Simulate the management rules on bars AFTER entry_pos.
    Returns a dict with realized return, flags (tp1/2/3 hit), and exit info.
    """
    # Levels
    tp1 = ph2_price
    tp2 = (ph2_price - pl1_price) * 1.618 + pl1_price
    tp3 = (ph2_price - pl1_price) * 2.618 + pl1_price

    # Initial stop: average of PL1 and support
    stop = (pl1_price + support_level) / 2.0

    # Position sizing (total = 1.0 “all capital”)
    remaining = 1.0
    realized = 0.0

    # Flags
    hit_tp1 = False
    hit_tp2 = False
    hit_tp3 = False

    # After TP1, stop moves to ENTRY; after TP2, stop moves to PH2
    n = len(df)
    for pos in range(entry_pos + 1, n):
        low = float(df.iloc[pos]['low'])
        high = float(df.iloc[pos]['high'])
        time = df.iloc[pos]['timestamp'] if 'timestamp' in df.columns else pos

        # SEQUENCE: for a long, evaluate downside (stop) before upside (TP) per bar
        # (conservative fill assumption)
        if not hit_tp1:
            # Stop check
            if low <= stop:
                # exit ALL at stop
                realized += remaining * ((stop / entry_price) - 1.0)
                remaining = 0.0
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': stop,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }
            # TP1 check
            if high >= tp1:
                # sell 50%
                sell_frac = 0.5 * remaining
                realized += sell_frac * ((tp1 / entry_price) - 1.0)
                remaining -= sell_frac
                hit_tp1 = True
                # move stop to entry
                stop = entry_price
                # continue to next bar
                continue

        elif hit_tp1 and not hit_tp2:
            # Stop @ entry
            if low <= stop:
                # exit remaining at entry (breakeven)
                realized += remaining * ((stop / entry_price) - 1.0)
                remaining = 0.0
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': stop,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }
            # TP2
            if high >= tp2:
                sell_frac = 0.5 * remaining  # half of the remainder
                realized += sell_frac * ((tp2 / entry_price) - 1.0)
                remaining -= sell_frac
                hit_tp2 = True
                # move stop to PH2
                stop = ph2_price
                continue

        elif hit_tp2 and not hit_tp3:
            # Stop @ PH2
            if low <= stop:
                # exit remaining at PH2
                realized += remaining * ((stop / entry_price) - 1.0)
                remaining = 0.0
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': stop,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }
            # TP3
            if high >= tp3:
                # exit ALL remaining
                realized += remaining * ((tp3 / entry_price) - 1.0)
                remaining = 0.0
                hit_tp3 = True
                return {
                    'exit_pos': pos, 'exit_time': time, 'exit_price': tp3,
                    'ret': realized,
                    'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
                }

    # If we reach here: no exit rule hit before data ended — close at last close
    last_close = float(df.iloc[-1]['close'])
    realized += remaining * ((last_close / entry_price) - 1.0)
    return {
        'exit_pos': len(df) - 1,
        'exit_time': df.iloc[-1]['timestamp'] if 'timestamp' in df.columns else len(df) - 1,
        'exit_price': last_close,
        'ret': realized,
        'hit_tp1': hit_tp1, 'hit_tp2': hit_tp2, 'hit_tp3': hit_tp3
    }

def backtest_liquidity_fvg(out_df: pd.DataFrame, setups: list):
    """
    Run the trade simulation across all setups (in chronological order).
    Returns (trades_df, summary_stats).
    """
    if not setups:
        return pd.DataFrame(), {
            'final_return': 0.0, 'average_return': 0.0, 'success_rate': 0.0,
            'tp1_hits': 0, 'tp2_hits': 0, 'tp3_hits': 0, 'max_drawdown': 0.0,
            'n_trades': 0
        }

    # Ensure chronological order by PH2 (confirmation) time/pos
    df = out_df.sort_values('timestamp').reset_index(drop=True)
    enriched = []
    for s in setups:
        # Positions are positional (from earlier code). Safety clamps:
        ph2_pos = int(s['ph2_pos'])
        pl1_pos = int(s['pl1_pos'])
        if ph2_pos >= len(df) or pl1_pos >= len(df):  # skip out-of-range
            continue

        entry_price = s['buy_price_0618']
        entry_pos, entry_time = _find_entry_after_ph2(df, ph2_pos, entry_price)
        if entry_pos is None:
            enriched.append({
                'status': 'no_entry',
                'entry_time': None, 'entry_pos': None, 'entry_price': entry_price,
                'pl1_price': s['pl1_price'],
                'support_level': s['support_level'],
                'ph2_price': s['ph2_price'],
                'ret': 0.0,
                'hit_tp1': False, 'hit_tp2': False, 'hit_tp3': False
            })
            continue

        # simulate
        result = _simulate_trade_long(
            df=df,
            entry_pos=entry_pos,
            entry_price=entry_price,
            pl1_price=s['pl1_price'],
            support_level=s['support_level'],
            ph2_price=s['ph2_price']
        )

        enriched.append({
            'status': 'filled',
            'entry_time': entry_time, 'entry_pos': entry_pos, 'entry_price': entry_price,
            'pl1_price': s['pl1_price'],
            'support_level': s['support_level'],
            'ph2_price': s['ph2_price'],
            'exit_time': result['exit_time'], 'exit_pos': result['exit_pos'], 'exit_price': result['exit_price'],
            'ret': result['ret'],
            'hit_tp1': result['hit_tp1'], 'hit_tp2': result['hit_tp2'], 'hit_tp3': result['hit_tp3']
        })

    trades_df = pd.DataFrame(enriched)

    # Keep only executed trades for stats (status == 'filled')
    exec_df = trades_df[trades_df['status'] == 'filled'].copy()
    n_trades = len(exec_df)

    if n_trades == 0:
        return trades_df, {
            'final_return': 0.0, 'average_return': 0.0, 'success_rate': 0.0,
            'tp1_hits': 0, 'tp2_hits': 0, 'tp3_hits': 0, 'max_drawdown': 0.0,
            'n_trades': 0
        }

    # Returns are per-trade percentage returns (e.g., 0.12 = +12%)
    avg_ret = float(exec_df['ret'].mean())
    success_rate = float((exec_df['ret'] > 0).mean())

    # Count how many trades hit each TP at any time in the lifecycle
    tp1_hits = int(exec_df['hit_tp1'].sum())
    tp2_hits = int(exec_df['hit_tp2'].sum())
    tp3_hits = int(exec_df['hit_tp3'].sum())

    # Equity curve (compounded, sequential) for max drawdown
    # Assume we reinvest all equity each trade in chronological order of entry_time.
    exec_df = exec_df.sort_values('entry_time')
    equity = [1.0]
    for r in exec_df['ret']:
        equity.append(equity[-1] * (1.0 + r))
    equity = np.array(equity[1:])  # drop initial 1.0 placeholder

    if len(equity) == 0:
        mdd = 0.0
        final_ret = 0.0
    else:
        peaks = np.maximum.accumulate(equity)
        drawdowns = (equity - peaks) / peaks
        mdd = float(drawdowns.min()) if len(drawdowns) else 0.0
        final_ret = float(equity[-1] - 1.0)

    summary = {
        'final_return': final_ret,        # compounded return across trades
        'average_return': avg_ret,        # mean per-trade return
        'success_rate': success_rate,     # fraction of profitable trades
        'tp1_hits': tp1_hits,
        'tp2_hits': tp2_hits,
        'tp3_hits': tp3_hits,
        'max_drawdown': mdd,              # negative value (e.g., -0.23 = -23%)
        'n_trades': n_trades
    }

    return trades_df, summary

In [173]:
trades_df, summary = backtest_liquidity_fvg(out_df, setups)

print("===== Summary =====")
for k, v in summary.items():
    if isinstance(v, float):
        print(f"{k:>16}: {v:.4f}")
    else:
        print(f"{k:>16}: {v}")

print("\n===== Sample trades =====")
print(trades_df.head(10))

===== Summary =====
    final_return: 0.3265
  average_return: 0.0006
    success_rate: 0.3959
        tp1_hits: 96
        tp2_hits: 46
        tp3_hits: 20
    max_drawdown: -0.2023
        n_trades: 591

===== Sample trades =====
   status          entry_time  entry_pos  entry_price  pl1_price  \
0  filled 2020-01-08 16:40:00     2312.0     0.002118   0.002078   
1  filled 2020-01-08 16:10:00     2306.0     0.002152   0.002143   
2  filled 2020-01-23 22:00:00     6696.0     0.002238   0.002225   
3  filled 2020-01-28 18:25:00     8093.0     0.002375   0.002350   
4  filled 2020-01-31 14:10:00     8906.0     0.002374   0.002354   
5  filled 2020-02-03 02:00:00     9624.0     0.002434   0.002427   
6  filled 2020-02-05 08:20:00    10276.0     0.002562   0.002540   
7  filled 2020-02-08 03:35:00    11083.0     0.002756   0.002721   
8  filled 2020-02-08 03:35:00    11083.0     0.002756   0.002721   
9  filled 2020-02-19 21:45:00    14387.0     0.002668   0.002654   

   support_level  

In [110]:
# After running your strategy:
print(f"Total tradable setups detected: {len(setups)}")

Total tradable setups detected: 0
